# Cat vs. Dog w/ VGG16 & VGG19

In [0]:
!pip install tensorflow-gpu==2.0.0-rc0

     |████████████████████████████████| 380.5MB 46kB/s 
     |████████████████████████████████| 501kB 55.1MB/s 
     |████████████████████████████████| 4.3MB 64.7MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, ZeroPadding2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD

print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt

2.2.0-rc2


In [0]:
!git clone https://github.com/laxmimerit/dog-cat-full-dataset.git

Cloning into 'dog-cat-full-dataset'...
remote: Enumerating objects: 25027, done.
remote: Counting objects: 100% (25027/25027), done.
remote: Compressing objects: 100% (25022/25022), done.
remote: Total 25027 (delta 5), reused 25020 (delta 3), pack-reused 0
Receiving objects: 100% (25027/25027), 541.62 MiB | 38.19 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (25001/25001), done.


In [0]:
test_data_dir = '/content/dog-cat-full-dataset/data/test'
train_data_dir = '/content/dog-cat-full-dataset/data/train'

In [0]:
img_width = 100
img_height = 100
batch_size = 10

In [0]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras import optimizers

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = datagen.flow_from_directory(directory=train_data_dir,
                                              target_size = (img_width, img_height),
                                              classes = ['dogs', 'cats'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

Found 20000 images belonging to 2 classes.


In [0]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [0]:
validation_generator = datagen.flow_from_directory(directory=test_data_dir,
                                                   target_size = (img_width, img_height),
                                                   classes = ['dogs', 'cats'],
                                                   class_mode = 'binary',
                                                   batch_size = batch_size)

Found 5000 images belonging to 2 classes.


# VGG16

In [0]:
model = Sequential()
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
# model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform', input_shape = (img_width, img_height, 3)))
# model.add(MaxPool2D(2,2))
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


model.compile(loss='poisson', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

model.summary()

NameError: ignored

In [0]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs = 5, validation_data=validation_generator, validation_steps=len(validation_generator), verbose = 1)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2000/2000 [==============================] - 851s 426ms/step - loss: 0.1652 - acc: 0.9304 - val_loss: 0.0980 - val_acc: 0.9606
Epoch 2/5
2000/2000 [==============================] - 845s 423ms/step - loss: 0.0716 - acc: 0.9729 - val_loss: 0.0981 - val_acc: 0.9634
Epoch 3/5
2000/2000 [==============================] - 845s 422ms/step - loss: 0.0422 - acc: 0.9853 - val_loss: 0.1148 - val_acc: 0.9626
Epoch 4/5
2000/2000 [==============================] - 844s 422ms/step - loss: 0.0220 - acc: 0.9926 - val_loss: 0.1125 - val_acc: 0.9680
Epoch 5/5
2000/2000 [==============================] - 842s 421ms/step - loss: 0.0127 - acc: 0.9962 - val_loss: 0.1142 - val_acc: 0.9688


# VGG19

In [0]:
model2 = Sequential()
conv_base2 = VGG19(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
# model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform', input_shape = (img_width, img_height, 3)))
# model.add(MaxPool2D(2,2))
model2.add(conv_base2)
model2.add(Flatten())
model2.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model2.add(Dense(1, activation='sigmoid'))

conv_base2.trainable = True

set_trainable = False
for layer in conv_base2.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


model2.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

model2.summary()

80142336/80134624 [==============================] - 2s 0us/step
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 24,218,945
Trainable params: 13,633,793
Non-trainable params: 10,585,152
_________________________________________________________________


In [0]:
history2 = model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs = 5, validation_data=validation_generator, validation_steps=len(validation_generator), verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 840s 420ms/step - loss: 0.0067 - acc: 0.9980 - val_loss: 0.1552 - val_acc: 0.9640
Epoch 2/5
2000/2000 [==============================] - 843s 422ms/step - loss: 0.0039 - acc: 0.9991 - val_loss: 0.1493 - val_acc: 0.9686
Epoch 3/5
2000/2000 [==============================] - 843s 421ms/step - loss: 0.0045 - acc: 0.9991 - val_loss: 0.1810 - val_acc: 0.9684
Epoch 4/5
2000/2000 [==============================] - 843s 421ms/step - loss: 0.0031 - acc: 0.9994 - val_loss: 0.2207 - val_acc: 0.9652
Epoch 5/5
2000/2000 [==============================] - 842s 421ms/step - loss: 0.0018 - acc: 0.9997 - val_loss: 0.2009 - val_acc: 0.9690
